In [1]:
#SQLite Db
import sqlite3
#dataframes
import pandas as pd

import numpy as np
#plots
import matplotlib.pyplot as plt
#lsq interpolation
import scipy.interpolate as si 

In [2]:
# Create the connection.
cnx = sqlite3.connect('data.db')

#Find the name of the table
res = cnx.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

turbine


In [3]:
#read sql
df = pd.read_sql_query("SELECT * FROM turbine", cnx)

In [4]:
#Split started column to two columns
df[['Date','Time']] = df.valuetime.str.split(" ",expand=True,)
df = df[['started','Date', 'Time', 'locname', 'plantno', 'power','wind']]

In [ ]:
#change time format
df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%H:%M:%S')
df

In [ ]:
#create a df with hours 
df1=pd.date_range("00:00:00", "23:00:00", freq="60min").strftime('%H:%M:%S')

#another times
#df2=pd.date_range("00:00:02", "23:00:02", freq="60min").strftime('%H:%M:%S')
#df3=pd.date_range("00:00:59", "23:00:59", freq="60min").strftime('%H:%M:%S')
#df_time=df1.append(df2)
#df_time=df_time.append(df3)

print(df1)

In [ ]:
#keep only row with time format tt:00:00
#df_hourly=pd.DataFrame()
#for a in df1:
    #df_hourly = df_hourly.append(df.loc[df['Time'] == a])
#df_hourly.head(n=5)
df_hourly=df

#print df
df_hourly.head(10)

In [ ]:
#separe date time columns
df_hourly["DateTime"] = df_hourly["Date"] + " " + df_hourly["Time"]

#order ascending by wind to apply interpolation
df_hourly=df_hourly.sort_values(by=['wind'], axis=0, ascending=True)

#positive values for power
df_hourly=df_hourly[(df_hourly.power >= 0.0)]

#print first 15 rows
df_hourly.head(n=15)


In [ ]:
#separe by plant no
df_hourly_1=df_hourly[(df_hourly.plantno == 1)]
df_hourly_2=df_hourly[(df_hourly.plantno == 2)]

#sort ascending
#df_hourly_2=df_hourly_2.sort_values(by=['power'])

#df_hourly_2=df_hourly_2[2:]

#choose one of the turbine
df_hourly=df_hourly_1

#sort again
df_hourly=df_hourly.sort_values(by=['wind'])

In [ ]:
plt.plot(df_hourly["wind"], df_hourly["power"],"o")
plt.show()

## LSQ Spline

source: http://vadym-pasko.com/blog/2015/03/06/spline-approx-scipy.html

In [ ]:
x=df_hourly["wind"].reset_index(drop=True)
\
y=df_hourly["power"].reset_index(drop=True)
wmin, wmax = min(x), max(x) 
pmin, pmax = min(y), max(y)

In [ ]:
#Define some control points and set some variables:
n = len(x)
plotpoints = 100

#set spline degree and find knot vector:
k = 2

#Construct uniform knot vector:
knotspace = range(n)
knots = si.InterpolatedUnivariateSpline(knotspace, knotspace, k=k).get_knots()
knots_full = np.concatenate(([knots[0]]*k, knots, [knots[-1]]*k))

#nknot - is the number of knots in the reduced knot vector (one without k repeating values at the endings).
nknot=5
knot_offset = (wmax - wmin)/(nknot + 1)
knots = np.linspace(knot_offset, wmax-knot_offset, nknot)

#define the vector of weights w in order to force our further approximations to pass close to the end points of the original data
wend = 3
num_points=len(x)
w = [wend] + [1]*(num_points-2) + [wend]

In [ ]:
#Instantiate LSQUnivariateSpline class using coordinates of the data points, the knot vector and the weight vector:
lsqspline = si.LSQUnivariateSpline(x, y, knots, k=k, w=w)

In [ ]:
# evaluate control points 
def getControlPoints(knots, k):
    n = len(knots) - 1 - k
    cx = np.zeros(n)
    for i in range(n):
        tsum = 0
        for j in range(1, k+1):
            tsum += knots[i+j]
        cx[i] = float(tsum)/k
    return cx

cp = getControlPoints(knots_full, k)

In [ ]:
#Get full-length knot vector, spline coefficients and coordinates of control points along the x-axis:
knots = lsqspline.get_knots()
knots_full = np.concatenate(([knots[0]]*k, knots, [knots[-1]]*k))
coeffs_p = lsqspline.get_coeffs()
coeffs_w = getControlPoints(knots_full, k)

In [ ]:
nsample = 100
xP = np.linspace(x[0], x[len(x) - 1], nsample)
yP = lsqspline(xP)

In [ ]:
#plot results
fig, ax = plt.subplots()
ax.plot(x, y, 'go', label='Turbine Data')
ax.plot(xP, yP, 'yo', label='Interpolated Data')

plt.xlabel("Wind speed (m/s)")
plt.ylabel("Power (kW)")

legend = ax.legend(loc='upper left', shadow=True, fontsize='medium')
plt.show()

In [ ]:
#save itnerpolated data to a df
di = pd.DataFrame({'Windspeed': xP,'Power': yP})

#positive values for power
di=di[(di.Power >= 0.0)]

di

### Met Eirean Data

In [ ]:
#meteirean data
cnx = sqlite3.connect('MetEirean.db')

#Find the name of the table
res = cnx.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

In [ ]:
me_full = pd.read_sql_query("SELECT * FROM met_eireann_com", cnx)
me=me_full[["period", "u100wind","v100wind"]]
me.head(5)
%store me

In [ ]:
me['windspeed'] = np.sqrt(me['u100wind'].pow(2) + me['v100wind'].pow(2))
#values greater than 1 for windspeed
me=me[(me.windspeed > 1.5) & (me.windspeed <15.5)]
me=me.sort_values(by=['windspeed'])
me.head(5)

## Interpolation

In [ ]:
w, p = di['Windspeed'].values, di['Power'].values

# Interpolate (w,p)
f = si.interp1d(w, p, kind='cubic')

# Apply interpolation curve to windspeed values in forecast to get power.
me['power'] = me['windspeed'].apply(f).round(3)
me

## Check Data

In [ ]:
df1 = me.loc[(me['windspeed'] > 2.8) & (me['windspeed'] < 3) , ['windspeed','power']]
df1=df1.rename(columns={'power':'Power_ME'})
df2 = df_hourly.loc[(df_hourly["wind"] > 2.8) & (df_hourly["wind"] < 3) , ['wind','power']]
df2=df2.rename(columns={'power':'Power_Turbine'})

#result = df1.append(df2, ignore_index=True, sort=False)
#result = pd.concat([df1, df2], axis=1)
print( "\033[1m"+"MetEirean Dataframe"+"\033[0m")
print(df1.head(10))
print("\n")
print( "\033[1m"+"Interpolated Data"+"\033[0m")
print(df2.head(10))
